In [6]:
import numpy as np

In [7]:
def unpickle(file):
    import cPickle
    fo = open(file, 'rb')
    dict = cPickle.load(fo)
    fo.close()
    return dict

In [20]:
label_names = unpickle('cifar-10-batches-py/batches.meta')
data = unpickle('cifar-10-batches-py/data_batch_1')
test = unpickle('cifar-10-batches-py/test_batch')

label_names = label_names['label_names']
images = data['data']
labels = data['labels']

im_test = test['data']
la_test = test['labels']

print len(images)
print len(labels)
print (label_names)

10000
10000
['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


# Methode naïve ( moyenne / pixel)

In [56]:
class_img = np.zeros([10,3072])
cpt_class = np.zeros([10])
for i, im in enumerate(images):
    c = labels[i]
    cpt_class[c] += 1
    for j, p in enumerate(im):
        class_img[c][j] += p
        
'''for c in range(10):
    for i, v in enumerate(class_img[c]):
        class_img[c][i] = v/cpt_class[c]'''

for c in range(10):
    class_img[c] = [i/cpt_class[c] for i in class_img[c]]

In [88]:
# compteur pour voir quelles sont les classes 
# reconnues le plus facilement par cette méthode
cpt_par_class = np.zeros([10,2]) 

err = 0.
succ = 0.

for i, im in enumerate(im_test):
    d = np.zeros([10])
    indMin = 0
    mini = 10000
    for j in range(10):
        d[j] = np.linalg.norm(im_test[i] - class_img[j])
    for k, el in enumerate(d):
        if mini > el:
            mini = el
            indMin = k
    if indMin == la_test[i]:
        succ +=1
        cpt_par_class[la_test[i]][0] += 1
    else:
        err += 1
        cpt_par_class[la_test[i]][1] += 1

In [91]:
print 'succes : ', (succ)/10000, '%, ', 'erreur : ', (err)/10000, '%'

succes :  0.277 %,  erreur :  0.723 %


In [101]:
print 'taux de réussite par classes :\n'
for i, l in enumerate(label_names):
    print l, ' : ', cpt_par_class[i][0]/cpt_par_class[i].sum()*100,'%'

taux de réussite par classes :

airplane  :  53.6 %
automobile  :  19.5 %
bird  :  10.8 %
cat  :  5.0 %
deer  :  10.9 %
dog  :  28.8 %
frog  :  53.9 %
horse  :  17.9 %
ship  :  36.0 %
truck  :  40.6 %


On peux supposer que certaines classes sont plus facilement identifiables car ils ont des couleurs spécifiques (comme les avions, les grenouilles).
Contrairement aux autres qui peuvent avoir des couleurs plus diversifiées. 

D'où la différence de réussite avec cette méthode.

# Methode naïve (k-means)